# Analysis of `trips-info.xml`

In [5]:
import pprint
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

from paths import normal, scenarios

def get_trips(file_path):
    class SAXParser:
        def __init__(self):
            self.trips = []
            self.current_trip = {}

        def start(self, tag, attr):
            if tag == "tripinfo":
                self.current_trip["id"] = attr["id"]
                self.current_trip["duration"] = float(attr["duration"])
                self.current_trip["routeLength"] = float(attr["routeLength"])
                self.current_trip["rerouteNo"] = float(attr["rerouteNo"])

            if tag == "emissions":
                self.current_trip["emissions"] = attr

        def end(self, tag):
            if tag == "tripinfo":
                self.trips.append(self.current_trip)
                self.current_trip = {}

        def close(self):
            pass

    sax_parser = SAXParser()
    xml_parser = etree.XMLParser(target=sax_parser)
    etree.parse(file_path, parser=xml_parser)

    return sax_parser.trips

n_trips = get_trips(f"{normal}/trips-info.xml")
scenario_data = {}

for s in scenarios:
    trips = get_trips(f"{s}/trips-info.xml")
    scenario_data[s] = trips

### Mean distance and duration comparison for all vehicles

In [6]:
n_distances = np.array(list(map(lambda t: t["routeLength"], n_trips)))
n_durations = np.array(list(map(lambda t: t["duration"], n_trips)))

for s in scenarios:
    trips = scenario_data[s]
    distances = np.array(list(map(lambda t: t["routeLength"], trips)))
    durations = np.array(list(map(lambda t: t["duration"], trips)))
    print("Mean Trip Distance")
    print(f"{normal}:", n_distances.mean() / 1000, "km")
    print(f"{s}:", distances.mean() / 1000, "km")
    print("Difference:", (distances.mean() - n_distances.mean()) / 1000, "km", "\n")

    print("Mean Trip Duration")
    print(f"{normal}:", n_durations.mean() / 60, "mins")
    print(f"{s}:", durations.mean() / 60, "mins")
    print("Difference:", (durations.mean() - n_durations.mean()) / 60, "mins", "\n")

Mean Trip Distance
normal-output: 2.9940531725265735 km
apats-output: 3.5280345625511043 km
Difference: 0.5339813900245304 km 

Mean Trip Duration
normal-output: 6.249727446170619 mins
apats-output: 7.514690651403653 mins
Difference: 1.2649632052330342 mins 



### Mean distance/duration comparison only for rerouted vehicles

In [7]:
for s in scenarios:
    trips = scenario_data[s]

    rr_trips = list(filter(lambda t: t["rerouteNo"] > 0, trips))
    rr_distances = np.array(list(map(lambda t: t["routeLength"], rr_trips)))
    rr_durations = np.array(list(map(lambda t: t["duration"], rr_trips)))

    rr_ids = list(map(lambda t: t["id"], rr_trips))
    filtered_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))
    n_distances = np.array(list(map(lambda t: t["routeLength"], filtered_n_trips)))
    n_durations = np.array(list(map(lambda t: t["duration"], filtered_n_trips)))

    print("Mean Trip Distance")
    print(f"{normal}:", n_distances.mean() / 1000, "km")
    print(f"{s}:", rr_distances.mean() / 1000, "km")
    print("Difference:", (rr_distances.mean() - n_distances.mean()) / 1000, "km", "\n")

    print("Mean Trip Duration")
    print(f"{normal}:", n_durations.mean() / 60, "mins")
    print(f"{s}:", rr_durations.mean() / 60, "mins")
    print("Difference:", (rr_durations.mean() - n_durations.mean()) / 60, "mins", "\n")

Mean Trip Distance
normal-output: 3.723337022332506 km
apats-output: 5.343833325062035 km
Difference: 1.6204963027295294 km 

Mean Trip Duration
normal-output: 7.570140612076096 mins
apats-output: 11.395781637717123 mins
Difference: 3.8256410256410267 mins 



### Additional distance/duration for rerouted vehicles

In [4]:
for s in scenarios:
    trips = scenario_data[s]

    rr_trips = list(filter(lambda t: t["rerouteNo"] > 0, trips))
    rr_distances = np.array(list(map(lambda t: t["routeLength"], rr_trips)))
    rr_durations = np.array(list(map(lambda t: t["duration"], rr_trips)))

    rr_ids = list(map(lambda t: t["id"], rr_trips))
    filtered_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))

    additional_distances = []
    additional_durations = []
    for t in rr_trips:
        nt = list(filter(lambda nt: t["id"] == nt["id"], filtered_n_trips))[0]
        additional_distances.append(t["routeLength"] - nt["routeLength"])
        additional_durations.append(t["duration"] - nt["duration"])

    additional_distances = np.array(additional_distances)
    additional_durations = np.array(additional_durations)
    
    print("Avg additional distance:", additional_distances.mean() / 1000, "km")
    print("Avg additional duration:", additional_durations.mean() / 60, "mins")

Avg additional distance: 1.6204963027295285 km
Avg additional duration: 3.825641025641026 mins
